This code is copy pasted and modified from 
https://colab.research.google.com/github/janblechschmidt/PDEsByNNs/blob/main/DeepBSDE_Solver.ipynb#scrollTo=MkAvrqE461C-.

Can't make it do what I want it to do ...

In [ ]:
from time import time
import tensorflow as tf
import numpy as np

class DeepBSDE(tf.keras.Model):
    def __init__(self, 
                 t0=0.0,
                 t1=1.0,
                 dim=1,
                 X0=1.0,
                 time_steps=20,
                 mu = 0.07,
                 r=0.02,
                 sigma=0.2,
                 learning_rate=1e-2,
                 num_hidden_layers=2,
                 num_neurons=200,
                 DTYPE="float32",
                 **kwargs):
        """Set up basic architecture of deep BSDE NN model."""
        
        super().__init__(**kwargs)
        
        self.t0 = t0
        self.t1 = t1
        self.N = time_steps
        self.dim = dim
        self.r = r
        self.DTYPE = DTYPE
        self.mu = mu
        self.sigma = sigma
        self.x = X0*np.ones(self.dim)
        self.dt = (t1 - t0)/(self.N)
        self.sqrt_dt = np.sqrt(self.dt)
        self.learning_rate = learning_rate
        
        self.t_space = np.linspace(self.t0, self.t1, self.N + 1)[:-1]
        
        tf.keras.backend.set_floatx(self.DTYPE)
        # Set optimizer
        self.optimizer=tf.keras.optimizers.Adam(self.learning_rate,
                                                epsilon=1e-8)
        
        # Initialize value and gradient of u(t_0,X_{t_0}) by zeros
        #self.u0 = tf.Variable(np.zeros((1),dtype=DTYPE))
        #self.gradu0 = tf.Variable(np.zeros((1,self.dim),dtype=DTYPE))
        
        # Alternatively, initialize both randomly
        self.u0 = tf.Variable(np.random.uniform(10.0, 20.0,size=(1)).astype(self.DTYPE))
        self.gradu0 = tf.Variable(np.random.uniform(-1e-1, 1e-1, size=(1, self.dim)).astype(self.DTYPE))
        
        # Create template of dense layer without bias and activation
        _dense = lambda dim: tf.keras.layers.Dense(
            units=dim,
            activation=None,
            use_bias=False)
        
        # Create template of batch normalization layer
        _bn = lambda : tf.keras.layers.BatchNormalization(
            momentum=.99,
            epsilon=1e-6,
            beta_initializer=tf.random_normal_initializer(0.0, stddev=0.1),
            gamma_initializer=tf.random_uniform_initializer(0.1, 0.5))
        
        
        # Initialize a list of networks approximating the gradient of u(t, x) at t_i
        self.gradui = []
        
        # Loop over number of time steps
        for _ in range(self.N - 1):
            
            # Batch normalization on dim-dimensional input
            this_grad = tf.keras.Sequential()
            this_grad.add(tf.keras.layers.Input(shape = (self.dim,)))
            this_grad.add(_bn())
            
            # Hidden layers of type (Dense -> Batch Normalization -> ReLU)
            for _ in range(num_hidden_layers):
                this_grad.add(_dense(num_neurons))
                this_grad.add(_bn())
                this_grad.add(tf.keras.layers.ReLU())
                
            # Dense layer followed by batch normalization for output
            this_grad.add(_dense(self.dim))
            this_grad.add(_bn())
            self.gradui.append(this_grad)
      
            
    def draw_X_and_dW(self, num_sample):
        """ Method to draw num_sample paths of X. """
        
        # Draw increments of Brownian motion
        dW = np.random.normal(loc=0.0, scale=self.sqrt_dt, size=(num_sample, self.dim, self.N)).astype(self.DTYPE)
        
        # Initialize and set array of paths
        X = np.zeros((num_sample, self.dim, self.N+1), dtype=self.DTYPE)
        
        X[:, :, 0] = np.ones((num_sample, self.dim)) * self.x
        
        for i in range(self.N):
            # This corresponds to the Euler-Maruyama Scheme of GBM
            X[:, :, i+1] = X[:, :, i]* (1+ self.mu* self.dt + self.sigma * dW[:, :, i])
            
        # Return simulated paths as well as increments of Brownian motion
        return X, dW
    
    @tf.function
    def call(self, inp, training=False):
        """
        Method to perform one forward sweep through the network
        given inputs: inp - (X, dW)
                      training - boolean variable indicating training
        """
        X, dW = inp # uses same dW as X
        num_sample = X.shape[0]
        
        
        e_num_sample = tf.ones(shape=[num_sample, 1], dtype=self.DTYPE)
        
        # Value approximation at t0
        y = e_num_sample * self.u0
        
        # Gradient approximation at t0
        z = e_num_sample * self.gradu0
        
        for i in range(self.N-1):
            
            t = self.t_space[i]
            
            # Optimal control is attained by gradient
            eta1 = self.fun_f(t, X[:, :, i], y, z) * self.dt
            eta2 = -tf.reduce_sum(z * dW[:, :, i], axis=1, keepdims=True)
            # here we are using the same dW

            y = y + eta1 + eta2

            # New gradient approximation
            # The division by self.dim acts as a stabilizer
            z = self.gradui[i](X[:, :, i + 1], training=training) / self.dim

        # Final step
        eta1 = - self.fun_f(self.t_space[self.N-1], X[:, :, self.N-1], y, z) * self.dt
        eta2 = tf.reduce_sum(z * dW[:, :, self.N-1], axis=1, keepdims=True)

        y = y + eta1 + eta2

        return y
    
    def loss_fn(self, inputs, training=False):
        X, _ = inputs
        # Forward pass to compute value estimates
        y_pred = self.call(inputs, training)
        
        # Exact values at final time
        y = self.fun_g(X[:, :, -1])
        loss = tf.reduce_mean(tf.square(y-y_pred)) #MSE
        
        return loss 
    
    @tf.function
    def train(self, inp):
        loss, grad = self.grad(inp, training=True)
        self.optimizer.apply_gradients(zip(grad, self.trainable_variables))
        return loss
    
    @tf.function
    def grad(self, inputs, training=False):
        with tf.GradientTape() as tape:
            loss = self.loss_fn(inputs, training=training)
        grad = tape.gradient(loss, self.trainable_variables)
        return loss, grad
    
    def fun_f(self, t, x, y, z):
        raise NotImplementedError
        
    def fun_g(self, t, x, y, z):
        raise NotImplementedError
    
class AmericanPut(DeepBSDE):
    def __init__(self,K=1.0, **kwargs):
        super().__init__(**kwargs)
        self.K = K
    # driver function

    @tf.function
    def q(self, x, y):
        return self.r * self.K * tf.cast(y <= self.fun_g(x), dtype=self.DTYPE)

    @tf.function
    def fun_f(self, t, x, y, z):
        return -self.r*y + self.q(x,y) 

    # Terminal/Final cost or payoff
    @tf.function
    def fun_g(self, x):
        return tf.maximum(self.K - x, 0)
    

def experiment(model, sol,num_epochs=100,batch_size=100):
    # Initialize header
    print('  Iter        Loss        y   L1_rel    L1_abs   |   Time  Stepsize')
     
    # Init timer and history list
    t0 = time()
    history=[]
    for i in range(num_epochs):
        
        inp = model.draw_X_and_dW(batch_size)
        loss = model.train(inp)

        # Get current Y_0 \approx u(0,x)
        y = model.u0.numpy()[0]

        currtime = time() - t0
        l1abs = np.abs(y - sol)
        l1rel = l1abs / sol

        hentry = (i, loss.numpy(), y, l1rel, l1abs, currtime, model.learning_rate)
        history.append(hentry)
        if i%10 == 0:
            print('{:5d} {:12.4f} {:8.4f} {:8.4f}  {:8.4f}   | {:6.1f}  {:6.2e}'.format(*hentry))
    return history

In [ ]:
test = AmericanPut(K=110, 
                   X0=100, 
                   time_steps=5, 
                   dim=1, 
                   r= 0.02,
                   mu=0.07, 
                   t0 =0,
                   t1 = 1,
                   sigma=0.3, 
                   learning_rate=1e-2, 
                   num_hidden_layers=2, 
                   num_neurons=100)

In [ ]:
experiment(test, sol = 15.20, num_epochs=100, batch_size=100)